In [ ]:
#Data Wrangling for book recommendation system to be used by Kindle or Goodreads

In [1]:
#import pandas, matplotlib.pyplot, seaborn, datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import datetime
import os

In [2]:
#load data file
file = 'GoodReads_100k_books.csv'
book_data= pd.read_csv(file)

In [3]:
book_data.info

<bound method DataFrame.info of                                                   author bookformat  \
0                                   Laurence M. Hauptman  Hardcover   
1                       Charlotte Fiell,Emmanuelle Dirix  Paperback   
2                                          Andy Anderson  Paperback   
3                                   Carlotta R. Anderson  Hardcover   
4                                          Jean Leveille        NaN   
...                                                  ...        ...   
99995                                         Simon Monk      ebook   
99996  Tracie L. Miller-Nobles,Brenda L. Mattison,Ell...  Hardcover   
99997                                     C. John Miller  Paperback   
99998                                      Albert Marrin  Hardcover   
99999                                      Marc E. Fitch      ebook   

                                                    desc  \
0      Reveals that several hundred thousand Indians ..

In [4]:
#drop columns we do not need such as image
book_rategen = book_data.drop(columns= ['bookformat','desc','img','pages','reviews','isbn','isbn13'])
print(book_rategen.head())

                             author  \
0              Laurence M. Hauptman   
1  Charlotte Fiell,Emmanuelle Dirix   
2                     Andy Anderson   
3              Carlotta R. Anderson   
4                     Jean Leveille   

                                               genre  \
0  History,Military History,Civil War,American Hi...   
1          Couture,Fashion,Historical,Art,Nonfiction   
2                                   Politics,History   
3                                      Labor,History   
4                                                NaN   

                                                link  rating  \
0  https://goodreads.com/book/show/1001053.Betwee...    3.52   
1  https://goodreads.com/book/show/10010552-fashi...    4.51   
2  https://goodreads.com/book/show/1001077.Hungar...    4.15   
3  https://goodreads.com/book/show/1001079.All_Am...    3.83   
4  https://goodreads.com/book/show/10010880-les-o...    4.00   

                                           

In [5]:
#reorder columns
cols=list(book_rategen.columns.values)
first_col=book_rategen.pop('title')
book_rategen.insert(0, 'Title', first_col)
print(book_rategen)

                                                   Title  \
0      Between Two Fires: American Indians in the Civ...   
1                               Fashion Sourcebook 1920s   
2                                             Hungary 56   
3      All-American Anarchist: Joseph A. Labadie and ...   
4                                  Les oiseaux gourmands   
...                                                  ...   
99995  Make Your Own PCBs with Eagle: From Schematic ...   
99996       Horngren's Financial & Managerial Accounting   
99997  A Faith Worth Sharing: A Lifetime of Conversat...   
99998  A Volcano Beneath the Snow: John Brown's War A...   
99999  Paranormal Nation: Why America Needs Ghosts, U...   

                                                  author  \
0                                   Laurence M. Hauptman   
1                       Charlotte Fiell,Emmanuelle Dirix   
2                                          Andy Anderson   
3                                   Car

In [7]:
col2=list(book_rategen.columns.values)
last_col=book_rategen.pop('link')
book_rategen.insert(5, 'Link', last_col)
print(book_rategen)

['Title', 'author', 'genre', 'link', 'rating', 'totalratings']
                                                   Title  \
0      Between Two Fires: American Indians in the Civ...   
1                               Fashion Sourcebook 1920s   
2                                             Hungary 56   
3      All-American Anarchist: Joseph A. Labadie and ...   
4                                  Les oiseaux gourmands   
...                                                  ...   
99995  Make Your Own PCBs with Eagle: From Schematic ...   
99996       Horngren's Financial & Managerial Accounting   
99997  A Faith Worth Sharing: A Lifetime of Conversat...   
99998  A Volcano Beneath the Snow: John Brown's War A...   
99999  Paranormal Nation: Why America Needs Ghosts, U...   

                                                  author  \
0                                   Laurence M. Hauptman   
1                       Charlotte Fiell,Emmanuelle Dirix   
2                                   

In [8]:
#count and order missing data
#review missing title
missing_data= pd.concat([book_rategen.isnull().sum(), 100* book_rategen.isnull().mean()], axis=1)
missing_data.columns= ['count', '%']
missing_data.sort_values(by='count')

,count,%
author,0,0.000
rating,0,0.000
totalratings,0,0.000
Link,0,0.000
Title,1,0.001
genre,10467,10.467


In [9]:
#view columns with type 'object'
book_rategen.select_dtypes('object')

,Title,author,genre,Link
0,Between Two Fires: American Indians in the Civ...,Laurence M. Hauptman,"History,Military History,Civil War,American Hi...",https://goodreads.com/book/show/1001053.Betwee...
1,Fashion Sourcebook 1920s,"Charlotte Fiell,Emmanuelle Dirix","Couture,Fashion,Historical,Art,Nonfiction",https://goodreads.com/book/show/10010552-fashi...
2,Hungary 56,Andy Anderson,"Politics,History",https://goodreads.com/book/show/1001077.Hungar...
3,All-American Anarchist: Joseph A. Labadie and ...,Carlotta R. Anderson,"Labor,History",https://goodreads.com/book/show/1001079.All_Am...
4,Les oiseaux gourmands,Jean Leveille,NaN,https://goodreads.com/book/show/10010880-les-o...
...,...,...,...,...
99995,Make Your Own PCBs with Eagle: From Schematic ...,Simon Monk,NaN,https://goodreads.com/book/show/17320181-make-...
99996,Horngren's Financial & Managerial Accounting,"Tracie L. Miller-Nobles,Brenda L. Mattison,Ell...",NaN,https://goodreads.com/book/show/17320260-horng...
99997,A Faith Worth Sharing: A Lifetime of Conversat...,C. John Miller,"Christianity,Evangelism,Christian,Religion,The...",https://goodreads.com/book/show/173203.A_Faith...
99998,A Volcano Beneath the Snow: John Brown's War A...,Albert Marrin,"Nonfiction,History,Biography,Military History,...",https://goodreads.com/book/show/17320669-a-vol...


In [11]:
book_rategen['Title'].value_counts().head()

Title
Love in the Mask    32
Selected Poems      22
Cinderella          13
Redemption          11
Coming Home         11
Name: count, dtype: int64

In [12]:
book_rategen.describe()

,rating,totalratings
count,100000.000000,1.000000e+05
mean,3.833055,2.990764e+03
std,0.621237,3.635338e+04
min,0.000000,0.000000e+00
25%,3.660000,3.100000e+01
50%,3.910000,1.460000e+02
75%,4.140000,7.440000e+02
max,5.000000,3.819326e+06


In [13]:
book_rategen[['Title','author']].nunique

<bound method DataFrame.nunique of                                                    Title  \
0      Between Two Fires: American Indians in the Civ...   
1                               Fashion Sourcebook 1920s   
2                                             Hungary 56   
3      All-American Anarchist: Joseph A. Labadie and ...   
4                                  Les oiseaux gourmands   
...                                                  ...   
99995  Make Your Own PCBs with Eagle: From Schematic ...   
99996       Horngren's Financial & Managerial Accounting   
99997  A Faith Worth Sharing: A Lifetime of Conversat...   
99998  A Volcano Beneath the Snow: John Brown's War A...   
99999  Paranormal Nation: Why America Needs Ghosts, U...   

                                                  author  
0                                   Laurence M. Hauptman  
1                       Charlotte Fiell,Emmanuelle Dirix  
2                                          Andy Anderson  
3       

In [15]:
#want to look at multiple entries with the same title, check if they have the same author
#groupby may be a good choice
groupedtitle = book_rategen.groupby('Title')
groupedtitle.head()

,Title,author,genre,rating,totalratings,Link
0,Between Two Fires: American Indians in the Civ...,Laurence M. Hauptman,"History,Military History,Civil War,American Hi...",3.52,33,https://goodreads.com/book/show/1001053.Betwee...
1,Fashion Sourcebook 1920s,"Charlotte Fiell,Emmanuelle Dirix","Couture,Fashion,Historical,Art,Nonfiction",4.51,41,https://goodreads.com/book/show/10010552-fashi...
2,Hungary 56,Andy Anderson,"Politics,History",4.15,26,https://goodreads.com/book/show/1001077.Hungar...
3,All-American Anarchist: Joseph A. Labadie and ...,Carlotta R. Anderson,"Labor,History",3.83,6,https://goodreads.com/book/show/1001079.All_Am...
4,Les oiseaux gourmands,Jean Leveille,NaN,4.00,1,https://goodreads.com/book/show/10010880-les-o...
...,...,...,...,...,...,...
99995,Make Your Own PCBs with Eagle: From Schematic ...,Simon Monk,NaN,4.07,56,https://goodreads.com/book/show/17320181-make-...
99996,Horngren's Financial & Managerial Accounting,"Tracie L. Miller-Nobles,Brenda L. Mattison,Ell...",NaN,4.05,38,https://goodreads.com/book/show/17320260-horng...
99997,A Faith Worth Sharing: A Lifetime of Conversat...,C. John Miller,"Christianity,Evangelism,Christian,Religion,The...",4.27,122,https://goodreads.com/book/show/173203.A_Faith...
99998,A Volcano Beneath the Snow: John Brown's War A...,Albert Marrin,"Nonfiction,History,Biography,Military History,...",3.63,156,https://goodreads.com/book/show/17320669-a-vol...


In [16]:
book_sorts=book_rategen.sort_values('author')
print(book_sorts)

                                                   Title  \
64289  Complete Palmist A Practical Guide to the Stud...   
67606  Der Grosse Ploetz: Die Chronik Zur Weltgeschichte   
35626  The Wrath of Fate: Book 1 of The Airship Pirat...   
48952                                         Guilty Men   
99396                                              Light   
...                                                  ...   
52687                                 Ai lÃ  ai cá»§a ai   
85678                  ã•ãã‚‰è˜ã®ãƒšãƒƒãƒˆãªå½¼å¥³   
86209                                 æœˆå…‰ [Gekkou] SS   
40129                                             æœˆå…‰   
31555                                     marriage black   

                                                  author  \
64289                                            "Niblo"   
67606                            &. Ruprecht Vandenhoeck   
35626                             'Captain' Robert Brown   
48952        'Cato',Michael Foot,Peter 